In [1]:
!pip install anndata

In [9]:
import pandas as pd

bulk_feature_counts = pd.read_csv("/Users/kpax/Documents/aep/study/MSC/lab/PPMI_Project_133_RNASeq/ppmi_counts_meta_dataset.csv", index_col=0)

In [10]:
patient_status = pd.read_csv("/Users/kpax/Documents/aep/study/MSC/lab/PPMI_Project_133_RNASeq//Participant_Status_19Mar2025.csv")

In [11]:
genetic_mapping = {
    'ENRLPINK1': 'PINK1',
    'ENRLPRKN': 'PRKN',
    'ENRLSRDC': 'SRDC',
    'ENRLHPSM': 'HPSM',
    'ENRLRBD': 'RBD',
    'ENRLLRRK2': 'LRRK2',
    'ENRLSNCA': 'SNCA',
    'ENRLGBA': 'GBA'
}

def get_genetic_group(row):
    for col, group in genetic_mapping.items():
        if row[col] == 1:
            return group
    return 'None'

genetic_groups = patient_status[['PATNO'] + list(genetic_mapping.keys())].copy()
genetic_groups['Genetic_Group'] = genetic_groups.apply(get_genetic_group, axis=1)

bulk_feature_counts = bulk_feature_counts.merge(
    genetic_groups[['PATNO', 'Genetic_Group']],
    left_on='Patient',
    right_on='PATNO',
    how='left'
).drop(columns=['PATNO'])

bulk_feature_counts['Genetic_Group'] = bulk_feature_counts['Genetic_Group'].fillna('Unknown')

In [12]:
def map_age_group(age):
    if 30 <= age <= 50:
        return "30-50"
    elif 50 < age <= 70:
        return "50-70"
    elif 70 < age <= 80:
        return "70-80"
    elif age > 80:
        return ">80"
    return "Unknown"

patient_status['Age_Group'] = patient_status['ENROLL_AGE'].apply(map_age_group)

age_group_mapping = patient_status[['PATNO', 'Age_Group']]

bulk_feature_counts = bulk_feature_counts.merge(
    age_group_mapping,
    left_on='Patient',
    right_on='PATNO',
    how='left'
).drop(columns=['PATNO'])

In [22]:
import anndata as ad
import numpy as np

ppmi_adata = ad.AnnData(bulk_feature_counts.loc[:, bulk_feature_counts.columns.str.startswith("ENSG")])
ppmi_adata.obs['Sample'] = bulk_feature_counts['Sample'].values
ppmi_adata.obs['Diagnosis'] = bulk_feature_counts['Diagnosis'].values
ppmi_adata.obs['Visit'] = bulk_feature_counts['Visit'].values
ppmi_adata.obs['Gender'] = bulk_feature_counts['Gender'].values
ppmi_adata.obs['Patient'] = bulk_feature_counts['Patient'].values
ppmi_adata.obs['Genetic_Group'] = bulk_feature_counts['Genetic_Group'].values
ppmi_adata.obs['Age_Group'] = bulk_feature_counts['Age_Group'].values
ppmi_adata.layers['counts_log2'] = np.log2(ppmi_adata.X + 1)

ppmi_adata.write_h5ad("/Users/kpax/Documents/aep/study/MSC/lab/PPMI_Project_133_RNASeq/ppmi_adata.h5ad")

/opt/anaconda3/lib/python3.12/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
